## Problem Statement 

You need to build a model that is able to classify customer complaints based on the products/services. By doing so, you can segregate these tickets into their relevant categories and, therefore, help in the quick resolution of the issue.

You will be doing topic modelling on the <b>.json</b> data provided by the company. Since this data is not labelled, you need to apply NMF to analyse patterns and classify tickets into the following five clusters based on their products/services:

* Credit card / Prepaid card

* Bank account services

* Theft/Dispute reporting

* Mortgages/loans

* Others 


With the help of topic modelling, you will be able to map each ticket onto its respective department/category. You can then use this data to train any supervised model such as logistic regression, decision tree or random forest. Using this trained model, you can classify any new customer complaint support ticket into its relevant department.

## Pipelines that needs to be performed:

You need to perform the following eight major tasks to complete the assignment:

1.  Data loading

2. Text preprocessing

3. Exploratory data analysis (EDA)

4. Feature extraction

5. Topic modelling 

6. Model building using supervised learning

7. Model training and evaluation

8. Model inference

## Importing the necessary libraries

In [43]:
import json 
import numpy as np
import pandas as pd
import re, nltk, spacy, string
import en_core_web_sm
nlp = en_core_web_sm.load()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [44]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [62]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

## Loading the data

The data is in JSON format and we need to convert it to a dataframe.

In [45]:
# Opening JSON file 
f = open('/kaggle/input/automatic-ticket-classification/complaints-2021-05-14_08_16.json')# Write the path to your data file and load it 
  
# returns JSON object as  
# a dictionary 
data = json.load(f)
df=pd.json_normalize(data)

## Data preparation

In [46]:
# Inspect the dataframe to understand the given data.
df.sample(10)


,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
70234,complaint-public-v2,complaint,2001653,0.0,None,75083,2001653,Billing disputes,2016-07-08T12:00:00-05:00,TX,...,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2016-07-08T12:00:00-05:00,None,None,Yes,"I paid {$980.00} for a website, logo, & web ho...",None,Consent provided
51784,complaint-public-v2,complaint,864007,0.0,Older American,33069,864007,Other,2014-05-21T12:00:00-05:00,FL,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2014-05-21T12:00:00-05:00,None,None,Yes,,None,N/A
41991,complaint-public-v2,complaint,1556590,0.0,None,945XX,1556590,Closing/Cancelling account,2015-09-09T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2015-09-09T12:00:00-05:00,None,None,Yes,"Disputed last pd date in XXXX, account charged...",None,Consent provided
73080,complaint-public-v2,complaint,502723,0.0,None,11763,502723,Repaying your loan,2013-08-23T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2013-08-26T12:00:00-05:00,None,Non-federal student loan,Yes,,None,N/A
9616,complaint-public-v2,complaint,505168,0.0,None,02301,505168,False statements or representation,2013-08-27T12:00:00-05:00,MA,...,Closed with explanation,JPMORGAN CHASE & CO.,Phone,2013-08-28T12:00:00-05:00,None,Credit card,Yes,,Attempted to collect wrong amount,N/A
52465,complaint-public-v2,complaint,389746,0.0,None,95050,389746,Credit reporting,2013-04-23T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2013-04-24T12:00:00-05:00,None,None,Yes,,None,N/A
39871,complaint-public-v2,complaint,3024448,0.0,None,47122,3024448,Managing an account,2018-09-19T12:00:00-05:00,IN,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2018-09-20T12:00:00-05:00,None,Checking account,Yes,,Banking errors,N/A
10629,complaint-public-v2,complaint,3558437,0.0,None,301XX,3558437,Opening an account,2020-03-07T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2020-03-07T12:00:00-05:00,None,Checking account,Yes,I went in Chase Bank last year in attempt to o...,Unable to open an account,Consent provided
28750,complaint-public-v2,complaint,3542666,0.0,None,77583,3542666,Closing an account,2020-02-21T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2020-02-24T12:00:00-05:00,None,Checking account,Yes,,Company closed your account,N/A
23095,complaint-public-v2,complaint,564117,0.0,None,59105,564117,Advertising and marketing,2013-10-20T12:00:00-05:00,MT,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2013-10-22T12:00:00-05:00,None,None,Yes,,None,N/A


In [47]:
df.shape

(78313, 22)

In [48]:
#print the column names
df.columns

Index(['_index', '_type', '_id', '_score', '_source.tags', '_source.zip_code',
       '_source.complaint_id', '_source.issue', '_source.date_received',
       '_source.state', '_source.consumer_disputed', '_source.product',
       '_source.company_response', '_source.company', '_source.submitted_via',
       '_source.date_sent_to_company', '_source.company_public_response',
       '_source.sub_product', '_source.timely',
       '_source.complaint_what_happened', '_source.sub_issue',
       '_source.consumer_consent_provided'],
      dtype='object')

In [49]:
#Assign new column names
for column in df.columns:
    new_column = column.replace("_","").replace("source.", "")
    df.rename(columns={column: new_column}, inplace=True)

In [50]:
df.columns

Index(['index', 'type', 'id', 'score', 'tags', 'zipcode', 'complaintid',
       'issue', 'datereceived', 'state', 'consumerdisputed', 'product',
       'companyresponse', 'company', 'submittedvia', 'datesenttocompany',
       'companypublicresponse', 'subproduct', 'timely',
       'complaintwhathappened', 'subissue', 'consumerconsentprovided'],
      dtype='object')

In [51]:
df.sample(10)

,index,type,id,score,tags,zipcode,complaintid,issue,datereceived,state,...,companyresponse,company,submittedvia,datesenttocompany,companypublicresponse,subproduct,timely,complaintwhathappened,subissue,consumerconsentprovided
6009,complaint-public-v2,complaint,3996578,0.0,None,863XX,3996578,Closing an account,2020-12-04T12:00:00-05:00,AZ,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2020-12-07T12:00:00-05:00,None,Checking account,Yes,,Funds not received from closed account,N/A
46900,complaint-public-v2,complaint,361132,0.0,None,95616,361132,"Making/receiving payments, sending money",2013-03-20T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2013-03-22T12:00:00-05:00,None,Checking account,Yes,,None,N/A
74930,complaint-public-v2,complaint,393444,0.0,None,60409,393444,Other,2013-04-26T12:00:00-05:00,IL,...,Closed with explanation,JPMORGAN CHASE & CO.,Postal mail,2013-05-03T12:00:00-05:00,None,Other mortgage,Yes,,None,N/A
6430,complaint-public-v2,complaint,4089199,0.0,None,972XX,4089199,Incorrect information on your report,2021-01-24T12:00:00-05:00,OR,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2021-02-09T12:00:00-05:00,None,Credit reporting,Yes,There are unknown hard inquiries on my credit ...,Information belongs to someone else,Consent provided
31891,complaint-public-v2,complaint,2807487,0.0,None,923XX,2807487,Incorrect information on your report,2018-02-07T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2018-02-07T12:00:00-05:00,None,Credit reporting,Yes,,Account information incorrect,Other
39331,complaint-public-v2,complaint,354126,0.0,None,92130,354126,"Loan modification,collection,foreclosure",2013-03-14T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2013-03-14T12:00:00-05:00,None,Home equity loan or line of credit,Yes,,None,N/A
46539,complaint-public-v2,complaint,405090,0.0,None,32526,405090,"Loan modification,collection,foreclosure",2013-05-12T12:00:00-05:00,FL,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2013-05-14T12:00:00-05:00,None,FHA mortgage,Yes,,None,N/A
25792,complaint-public-v2,complaint,226323,0.0,"Older American, Servicemember",11714,226323,"Loan modification,collection,foreclosure",2013-01-09T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Phone,2013-01-09T12:00:00-05:00,None,Conventional fixed mortgage,Yes,,None,N/A
53565,complaint-public-v2,complaint,59241,0.0,None,32901,59241,Deposits and withdrawals,2012-04-17T12:00:00-05:00,FL,...,Closed with relief,JPMORGAN CHASE & CO.,Referral,2012-04-17T12:00:00-05:00,None,Checking account,Yes,,None,N/A
11486,complaint-public-v2,complaint,3735363,0.0,"Older American, Servicemember",81007,3735363,Problem with a purchase shown on your statement,2020-07-08T12:00:00-05:00,CO,...,Closed with explanation,JPMORGAN CHASE & CO.,Phone,2020-07-13T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Credit card company isn't resolving a dispute ...,N/A


In [52]:
(df.complaintwhathappened == "").sum()

57241

In [53]:
#Assign nan in place of blanks in the complaints column
df[df.complaintwhathappened == ""] = np.nan 

In [54]:
(df.complaintwhathappened == "").sum()

0

In [55]:
(df.complaintwhathappened == np.nan).sum()

0

In [56]:
#Remove all rows where complaints column is nan
df.dropna(subset=['complaintwhathappened'], inplace=True)

In [57]:
df.shape

(21072, 22)

## Prepare the text for topic modeling

Once you have removed all the blank complaints, you need to:

* Make the text lowercase
* Remove text in square brackets
* Remove punctuation
* Remove words containing numbers


Once you have done these cleaning operations you need to perform the following:
* Lemmatize the texts
* Extract the POS tags of the lemmatized text and remove all the words which have tags other than NN[tag == "NN"].


In [58]:
# Write your function here to clean the text and remove all the unnecessary elements.
def clean_text(text):
    # Make the text lowercase
    text = text.lower()
    
    # Remove text in square brackets using regular expression
    text = re.sub(r'\[.*?\]', '', text)
    
    # Remove punctuation using string library
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove words containing numbers using regular expression
    text = re.sub(r'\w*\d\w*', '', text)
    
    return text

In [59]:
#Write your function to Lemmatize the texts
def lemmatize_text(text):
    # Tokenize the text into words
    words = word_tokenize(text.lower())
    
    # Initialize the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Lemmatize each word using its part of speech (POS)
    lemmatized_words = []
    for word, pos in nltk.pos_tag(words):
        pos_letter = pos[0].lower() if pos[0].lower() in ['a', 'r', 'n', 'v'] else 'n'
        lemma = lemmatizer.lemmatize(word, pos=pos_letter)
        lemmatized_words.append(lemma)
    
    # Join the lemmatized words back into a sentence
    lemmatized_text = ' '.join(lemmatized_words)
    
    return lemmatized_text

In [60]:
df['cleaned_complaints'] = df['complaintwhathappened'].apply(clean_text)

In [63]:
df['lemmatized_complaints'] = df['cleaned_complaints'].apply(lemmatize_text)

In [64]:
df['lemmatized_complaints'].sample(10)

15538    i have a home equity line of credit with chase...
17602    on xxxxxxxx last year i visit chase bank branc...
42888    i use to have this credit card chase freedom a...
43138    this be my complaint against chase bank first ...
24566    on xxxx my computer be hack all credit card ba...
6063     this account be open fraudulently under the fc...
17260    i go to make a purchase for approx xxxx fro xx...
18785    computer be hack chase visa card be bill and w...
57242    xxxx xxxx xxxx xxxx xxxx xxxx xxxx ny xxxx xxx...
3911     i be file a complaint regard amazoncom and cha...
Name: lemmatized_complaints, dtype: object

In [65]:
#Create a dataframe('df_clean') that will have only the complaints and the lemmatized complaints 
df_clean = pd.DataFrame(columns=['complaintwhathappened','lemmatized_complaints'], data=df[['complaintwhathappened','lemmatized_complaints']])

In [66]:
df_clean

,complaintwhathappened,lemmatized_complaints
1,Good morning my name is XXXX XXXX and I apprec...,good morning my name be xxxx xxxx and i apprec...
2,I upgraded my XXXX XXXX card in XX/XX/2018 and...,i upgrade my xxxx xxxx card in and be tell by ...
10,Chase Card was reported on XX/XX/2019. However...,chase card be report on however fraudulent app...
11,"On XX/XX/2018, while trying to book a XXXX XX...",on while try to book a xxxx xxxx ticket i come...
14,my grand son give me check for {$1600.00} i de...,my grand son give me check for i deposit it in...
...,...,...
78303,After being a Chase Card customer for well ove...,after be a chase card customer for well over a...
78309,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",on wednesday xxxxxxxx i call chas my xxxx xxxx...
78310,I am not familiar with XXXX pay and did not un...,i be not familiar with xxxx pay and do not und...
78311,I have had flawless credit for 30 yrs. I've ha...,i have have flawless credit for yr ive have ch...


In [ ]:
#Write your function to extract the POS tags 

def pos_tag(text):
  # write your code here



df_clean["complaint_POS_removed"] =  #this column should contain lemmatized text with all the words removed which have tags other than NN[tag == "NN"].


In [ ]:
#The clean dataframe should now contain the raw complaint, lemmatized complaint and the complaint after removing POS tags.
df_clean

## Exploratory data analysis to get familiar with the data.

Write the code in this task to perform the following:

*   Visualise the data according to the 'Complaint' character length
*   Using a word cloud find the top 40 words by frequency among all the articles after processing the text
*   Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text. ‘




In [ ]:
# Write your code here to visualise the data according to the 'Complaint' character length

#### Find the top 40 words by frequency among all the articles after processing the text.

In [ ]:
#Using a word cloud find the top 40 words by frequency among all the articles after processing the text


In [ ]:
#Removing -PRON- from the text corpus
df_clean['Complaint_clean'] = df_clean['complaint_POS_removed'].str.replace('-PRON-', '')

#### Find the top unigrams,bigrams and trigrams by frequency among all the complaints after processing the text.

In [ ]:
#Write your code here to find the top 30 unigram frequency among the complaints in the cleaned datafram(df_clean). 


In [ ]:
#Print the top 10 words in the unigram frequency


In [ ]:
#Write your code here to find the top 30 bigram frequency among the complaints in the cleaned datafram(df_clean). 


In [ ]:
#Print the top 10 words in the bigram frequency

In [ ]:
#Write your code here to find the top 30 trigram frequency among the complaints in the cleaned datafram(df_clean). 


In [ ]:
#Print the top 10 words in the trigram frequency

## The personal details of customer has been masked in the dataset with xxxx. Let's remove the masked text as this will be of no use for our analysis

In [ ]:
df_clean['Complaint_clean'] = df_clean['Complaint_clean'].str.replace('xxxx','')

In [ ]:
#All masked texts has been removed
df_clean

## Feature Extraction
Convert the raw texts to a matrix of TF-IDF features

**max_df** is used for removing terms that appear too frequently, also known as "corpus-specific stop words"
max_df = 0.95 means "ignore terms that appear in more than 95% of the complaints"

**min_df** is used for removing terms that appear too infrequently
min_df = 2 means "ignore terms that appear in less than 2 complaints"

In [ ]:
#Write your code here to initialise the TfidfVectorizer 



#### Create a document term matrix using fit_transform

The contents of a document term matrix are tuples of (complaint_id,token_id) tf-idf score:
The tuples that are not there have a tf-idf score of 0

In [ ]:
#Write your code here to create the Document Term Matrix by transforming the complaints column present in df_clean.


## Topic Modelling using NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised technique so there are no labeling of topics that the model will be trained on. The way it works is that, NMF decomposes (or factorizes) high-dimensional vectors into a lower-dimensional representation. These lower-dimensional vectors are non-negative which also means their coefficients are non-negative.

In this task you have to perform the following:

* Find the best number of clusters 
* Apply the best number to create word clusters
* Inspect & validate the correction of each cluster wrt the complaints 
* Correct the labels if needed 
* Map the clusters to topics/cluster names

In [ ]:
from sklearn.decomposition import NMF

## Manual Topic Modeling
You need to do take the trial & error approach to find the best num of topics for your NMF model.

The only parameter that is required is the number of components i.e. the number of topics we want. This is the most crucial step in the whole topic modeling process and will greatly affect how good your final topics are.

In [ ]:
#Load your nmf_model with the n_components i.e 5
num_topics = #write the value you want to test out

#keep the random_state =40
nmf_model = #write your code here

In [ ]:
nmf_model.fit(dtm)
len(tfidf.get_feature_names())

In [ ]:
#Print the Top15 words for each of the topics


In [ ]:
#Create the best topic for each complaint in terms of integer value 0,1,2,3 & 4



In [ ]:
#Assign the best topic to each of the cmplaints in Topic Column

df_clean['Topic'] = #write your code to assign topics to each rows.

In [ ]:
df_clean.head()

In [ ]:
#Print the first 5 Complaint for each of the Topics
df_clean=df_clean.groupby('Topic').head(5)
df_clean.sort_values('Topic')

#### After evaluating the mapping, if the topics assigned are correct then assign these names to the relevant topic:
* Bank Account services
* Credit card or prepaid card
* Theft/Dispute Reporting
* Mortgage/Loan
* Others

In [ ]:
#Create the dictionary of Topic names and Topics

Topic_names = {   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

## Supervised model to predict any new complaints to the relevant Topics.

You have now build the model to create the topics for each complaints.Now in the below section you will use them to classify any new complaints.

Since you will be using supervised learning technique we have to convert the topic names to numbers(numpy arrays only understand numbers)

In [ ]:
#Create the dictionary again of Topic names and Topics

Topic_names = {   }
#Replace Topics with Topic Names
df_clean['Topic'] = df_clean['Topic'].map(Topic_names)

In [ ]:
df_clean

In [ ]:
#Keep the columns"complaint_what_happened" & "Topic" only in the new dataframe --> training_data
training_data=

In [ ]:
training_data

####Apply the supervised models on the training data created. In this process, you have to do the following:
* Create the vector counts using Count Vectoriser
* Transform the word vecotr to tf-idf
* Create the train & test data using the train_test_split on the tf-idf & topics


In [ ]:

#Write your code to get the Vector count


#Write your code here to transform the word vector to tf-idf

You have to try atleast 3 models on the train & test data from these options:
* Logistic regression
* Decision Tree
* Random Forest
* Naive Bayes (optional)

**Using the required evaluation metrics judge the tried models and select the ones performing the best**

In [ ]:
# Write your code here to build any 3 models and evaluate them using the required metrics



